In [120]:
import json
import numpy as np
import pandas as pd 
file_name = "user_data1.csv"
user_info = pd.read_csv(file_name)
#print(user_info.head())

# with open(file_name, encoding = 'utf-8') as json_file:
#     data = json.load(json_file)
#     print(len(data))
#报错由于json文件过大，计算机内存不够
#由于中文解码有点问题，以下变量转换成英文
#用户被认可度-acceptance
#用户活跃度-activation
#用户互动频率-frequency
#用户被关注度-subscribe
#注册年限-registeration

In [122]:
#用户被认可度-acceptance
acceptance = user_info.acceptance
cool = user_info.useful
review_count = user_info.review_count
acceptance = cool/review_count
user_info.acceptance = acceptance

#用户活跃度-activation
from dateutil.relativedelta import relativedelta
now_date = pd.to_datetime(user_info.now_date).astype(np.int64)
yelping_since = pd.to_datetime(user_info.yelping_since).astype(np.int64)
#days
date = (now_date-yelping_since)/(3600*24*1e9)
#years
years = date/365.25
user_info.registeration = years
review_count = user_info.review_count
activation = review_count / years
user_info.activation = activation

#用户互动频率-frequency
tag_count = (user_info.compliment_cool + user_info.compliment_cute
            + user_info.compliment_funny + user_info.compliment_hot 
            + user_info.compliment_list + user_info.compliment_more 
            + user_info.compliment_note + user_info.compliment_photos
            + user_info.compliment_plain + user_info.compliment_profile
            + user_info.compliment_writer)
like_count = user_info.useful + user_info.funny + user_info.cool
frequency = tag_count / like_count
user_info.frequency = frequency

#用户被关注度-subscribe
subscribe = 0.5 * like_count + 0.5 * user_info.fans
user_info.subscribe = subscribe
#print(user_info.head)

In [123]:
#最大特征值
R = np.array([[1, 6, 4, 2, 3, 8],
             [1/6, 1, 1/3, 1/5, 1/4, 3],
             [1/4, 3, 1, 1/4, 1/2, 4],
             [1/2, 5, 4, 1, 2, 7],
             [1/3, 4, 2, 1/2, 1, 5],
             [1/8, 1/3, 1/4, 1/7, 1/5, 1]])
lambda_array, W = np.linalg.eig(R)
lambda_max = np.max(lambda_array)
print(lambda_max)
m = 6
CI = (lambda_max - m) / (m - 1)
print(CI)
RI = 1.24
CR = CI / RI

def normalization(W):
    _range = np.max(W) - np.min(W)
    return (W - np.min(W)) / _range
W = normalization(W)
print(W)
Wi = np.empty([len(W), 1])
for i in range(len(W)):
    SUM = np.sum(W[i, :])
    Wi[i] = SUM
Wi = Wi/np.sum(Wi)
print(Wi)

(6.225751920039733+0j)
(0.045150384007946624+0j)
[[0.00509963+0.j         0.        +0.j         0.        -0.j
  1.        +0.j         1.        -0.j         0.54707424+0.j        ]
 [0.4220189 +0.j         0.55126828+0.06461402j 0.55126828-0.06461402j
  0.50183791+0.06351652j 0.50183791-0.06351652j 0.50017737+0.j        ]
 [0.36225541+0.j         0.62314401-0.05994708j 0.62314401+0.05994708j
  0.52347115-0.11065343j 0.52347115+0.11065343j 0.61134767+0.j        ]
 [0.14919566+0.j         0.3052584 -0.29162631j 0.3052584 +0.29162631j
  0.21529909+0.25038725j 0.21529909-0.25038725j 0.92552525+0.j        ]
 [0.28753362+0.j         0.536482  -0.15385997j 0.536482  +0.15385997j
  0.47384595-0.03222493j 0.47384595+0.03222493j 0.03162404+0.j        ]
 [0.45181461+0.j         0.46838546+0.0413759j  0.46838546-0.0413759j
  0.4691405 -0.02512542j 0.4691405 +0.02512542j 0.48101699+0.j        ]]
[[0.15841239]
 [0.18797208]
 [0.20277101]
 [0.13132907]
 [0.14523127]
 [0.17428418]]


<ipython-input-123-11d13079a26b>:25: ComplexWarning: Casting complex values to real discards the imaginary part
  Wi[i] = SUM


In [124]:
#熵值法
x = np.vstack((user_info.acceptance, user_info.activation, user_info.frequency, user_info.subscribe, user_info.fans, user_info.registeration))
def normalization_entropy(x):
    X = np.empty([len(x),len(x[0])])
    for i in range(len(x)):
        X[i, :] = (x[i, :] - np.min(x[i, :])) / (np.max(x[i, :]) - np.min(x[i, :])) 
    return X
X = normalization_entropy(x)
import math
def SUM_column(Q, i):
    SUM = 0
    for j in range(len(Q[0])):
        if Q[i, j] > 0:
            SUM += Q[i, j] * math.log(Q[i, j])
    return SUM
def rate_entropy(X):
    Q = np.empty([len(X),len(X[0])])
    for i in range(len(x)):
        for j in range(len(x[0])):
            Q[i, j] = X[i, j] / np.sum(X[i, :])
    e = np.empty([len(Q), 1])
    for i in range(len(x)):
        e[i] = -1/math.log(len(Q[0])) * SUM_column(Q, i)
    return e
e = np.empty([len(X), 1])
e = rate_entropy(X)
r = 1 - e
Wj = r / np.sum(r)
print(Wj)

[[0.19476501]
 [0.13611774]
 [0.12955741]
 [0.28790498]
 [0.23667792]
 [0.01497694]]


In [125]:
#综合权重赋值法
def distance(X, Y):
    distance_sq = 0
    for i in range(len(X)):
        distance_sq += (X[i] - Y[i]) * (X[i] - Y[i])
    distance_sq /= 0.5
    return distance_sq
distance_sq = distance(Wi, Wj)
#通过计算器解出两个解 alpha = 0.68388 beta = 0.31611, 或者 alpha = 0.31611 beta = 0.68388
W1 = 0.68388 * Wi + 0.31611 * Wj
W2 = 0.31611 * Wi + 0.68388 * Wj


In [126]:
#各种影响力导入CSV
effects_W1 = np.dot(X.T,W1)
effects_W2 = np.dot(X.T,W2)
effects_Wi = np.dot(X.T,Wi)
effects_Wj = np.dot(X.T,Wj)
user_info.W1 = effects_W1
user_info.W2 = effects_W2
user_info.Wi = effects_Wi
user_info.Wj = effects_Wj
print(user_info.head)


<bound method NDFrame.head of                      user_id  compliment_cool  compliment_cute  \
0     q_QQ5kBBwlCcbL1s4NVK3g             2541              361   
1     dIIKEfOgo0KqUfGQvGikPg             2205              232   
2     QJI9OSEn6ujRCtrX06vs1w             2090               52   
3     BwTPj8XS_iqdSTLPBVMs1w              250               87   
4     8ZryN_S-n48g6rsa3W3QtQ             4928              105   
...                      ...              ...              ...   
1830  F_5_UNX-wrAFCXuAkBZRDw             2916               54   
1831  jNsPTMmg9EcqY7q2Q-Rm6A              254               95   
1832  qPCUIYg0GzZAXPTGAr8iRA              213                4   
1833  sb1_3-c5LfqFVnnAeftLPg               56                0   
1834  yY5KBM05U8C8b1HjNkej7A              551               25   

      compliment_funny  compliment_hot  compliment_list  compliment_more  \
0                 2541            1710              147              163   
1                 2205   

In [127]:
user_info.to_csv("userinfo.csv",sep=',')